In [20]:
from dotenv import load_dotenv
import os
import json
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [21]:
import json

with open('../opentrons_config.json', 'r') as file:
    opentrons_config_string = file.read()

opentrons_config = json.loads(opentrons_config_string)
json.dumps(opentrons_config)

'{"robot_type": "OT-2", "api_version": "2.17", "pipettes": {"left": {"type": "p300_single_gen2", "tip_racks": ["1"]}, "right": {"type": "p20_multi_gen2", "tip_racks": ["2"]}}, "modules": {"thermocycler": {"type": "thermocyclerModuleV2", "location": ["7"]}}, "labware": {"1": {"type": "opentrons_96_tiprack_300ul"}, "2": {"type": "opentrons_96_tiprack_20ul"}, "3": {"type": "corning_96_wellplate_360ul_flat"}, "5": {"type": "nest_12_reservoir_15ml"}, "7": {"type": "nest_96_wellplate_100ul_pcr_full_skirt", "parent_module": "thermocycler"}}}'

In [22]:
from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field

class DeckConfig(BaseModel):
    config_string: str = Field(..., description="JSON string representation of the Opentrons deck layout")

def initialize_deck(config_string: str) -> str:
    """
    Initialize the Opentrons deck layout based on the provided JSON configuration.
    
    Args:
        config_string: JSON string representation of the deck layout.
    
    Returns:
        str: Python code to set up the Opentrons deck.
    """
    config = json.loads(config_string)
    code = []
    
    code.append("from opentrons import protocol_api")
    code.append(f"\nmetadata = {{'apiLevel': '{config['api_version']}'}}")
    code.append("\ndef run(protocol: protocol_api.ProtocolContext):")
    
    # Load pipettes
    for mount, pipette in config['pipettes'].items():
        tip_racks = [f"protocol.load_labware('{config['labware'][rack]['type']}', '{rack}')" 
                     for rack in pipette['tip_racks']]
        tip_racks_str = ", ".join(tip_racks)
        code.append(f"    {mount}_pipette = protocol.load_instrument('{pipette['type']}', '{mount}', tip_racks=[{tip_racks_str}])")
    
    # Load modules
    for module_name, module_info in config.get('modules', {}).items():
        code.append(f"    {module_name} = protocol.load_module('{module_info['type']}', '{module_info['location'][0]}')")
    
    # Load labware
    for slot, labware in config['labware'].items():
        if 'parent_module' in labware:
            code.append(f"    {slot}_labware = {labware['parent_module']}.load_labware('{labware['type']}')")
        else:
            code.append(f"    {slot}_labware = protocol.load_labware('{labware['type']}', '{slot}')")
    
    return "\n".join(code)

initialize_deck_tool = StructuredTool.from_function(
    func=initialize_deck,
    name="InitializeDeck",
    description="Initialize the Opentrons deck layout based on a JSON configuration string.",
    args_schema=DeckConfig
)


In [23]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

system_template = """
You are an AI assistant that initializes Opentrons deck layouts. Your task is to:
    1. Take the provided JSON configuration string.
    2. Use the 'Initialize Deck' tool to generate the Python code for setting up the Opentrons deck.
    3. Return the generated Python code.

    Only use the 'Initialize Deck' tool. Do not attempt to modify or add to the generated code.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(model, [initialize_deck_tool], prompt)

In [24]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=[initialize_deck_tool], verbose=True)

In [25]:
agent_executor.invoke({"input": f"Set the temperature of a thermocycler block to 20 degC.\n\nOpentrons deck config:\n{json.dumps(opentrons_config)}"})



> Entering new AgentExecutor chain...

Invoking: `InitializeDeck` with `{'config_string': '{"robot_type": "OT-2", "api_version": "2.17", "pipettes": {"left": {"type": "p300_single_gen2", "tip_racks": ["1"]}, "right": {"type": "p20_multi_gen2", "tip_racks": ["2"]}}, "modules": {"thermocycler": {"type": "thermocyclerModuleV2", "location": ["7"]}}, "labware": {"1": {"type": "opentrons_96_tiprack_300ul"}, "2": {"type": "opentrons_96_tiprack_20ul"}, "3": {"type": "corning_96_wellplate_360ul_flat"}, "5": {"type": "nest_12_reservoir_15ml"}, "7": {"type": "nest_96_wellplate_100ul_pcr_full_skirt", "parent_module": "thermocycler"}}}'}`


from opentrons import protocol_api

metadata = {'apiLevel': '2.17'}

def run(protocol: protocol_api.ProtocolContext):
    left_pipette = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[protocol.load_labware('opentrons_96_tiprack_300ul', '1')])
    right_pipette = protocol.load_instrument('p20_multi_gen2', 'right', tip_racks=[protocol.load_labwa

{'input': 'Set the temperature of a thermocycler block to 20 degC.\n\nOpentrons deck config:\n{"robot_type": "OT-2", "api_version": "2.17", "pipettes": {"left": {"type": "p300_single_gen2", "tip_racks": ["1"]}, "right": {"type": "p20_multi_gen2", "tip_racks": ["2"]}}, "modules": {"thermocycler": {"type": "thermocyclerModuleV2", "location": ["7"]}}, "labware": {"1": {"type": "opentrons_96_tiprack_300ul"}, "2": {"type": "opentrons_96_tiprack_20ul"}, "3": {"type": "corning_96_wellplate_360ul_flat"}, "5": {"type": "nest_12_reservoir_15ml"}, "7": {"type": "nest_96_wellplate_100ul_pcr_full_skirt", "parent_module": "thermocycler"}}}',
 'output': "```python\nfrom opentrons import protocol_api\n\nmetadata = {'apiLevel': '2.17'}\n\ndef run(protocol: protocol_api.ProtocolContext):\n    left_pipette = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[protocol.load_labware('opentrons_96_tiprack_300ul', '1')])\n    right_pipette = protocol.load_instrument('p20_multi_gen2', 'right', tip